In [1]:
import numpy as np
from bayspec.model.local import *
from bayspec import DataUnit, Data, Infer
from bayspec.util.significance import pgsig

In [2]:
model = ppl()
print(model)

ppl [add]
cutoff power law model with peak energy
╒════════╤═════════════╤═════════════╤═════════╕
│  cfg#  │  Component  │  Parameter  │  Value  │
╞════════╪═════════════╪═════════════╪═════════╡
│   1    │     ppl     │  redshift   │    0    │
╘════════╧═════════════╧═════════════╧═════════╛
╒════════╤═════════════╤═════════════╤═════════╤═════════════╤══════════╕
│  par#  │  Component  │  Parameter  │  Value  │    Prior    │  Frozen  │
╞════════╪═════════════╪═════════════╪═════════╪═════════════╪══════════╡
│   1    │     ppl     │  $\alpha$   │   -1    │ unif(-2, 2) │  False   │
├────────┼─────────────┼─────────────┼─────────┼─────────────┼──────────┤
│   2    │     ppl     │ log$E_{p}$  │    2    │ unif(0, 4)  │  False   │
├────────┼─────────────┼─────────────┼─────────┼─────────────┼──────────┤
│   3    │     ppl     │   log$A$    │   -1    │ unif(-6, 5) │  False   │
╘════════╧═════════════╧═════════════╧═════════╧═════════════╧══════════╛



In [3]:
me = DataUnit(
    src='./ME/me.src', 
    bkg='./ME/me.bkg', 
    rsp='./ME/me.rsp', 
    notc=[8, 900], 
    stat='ULpgstat', 
    grpg={'min_sigma': 3, 'max_bin': 10})

data = Data([('ME', me)])

In [4]:
infer = Infer([(data, model)])
print(infer)

╒════════╤══════════════╤═════════════╤═════════════╤═════════╕
│  cfg#  │  Expression  │  Component  │  Parameter  │  Value  │
╞════════╪══════════════╪═════════════╪═════════════╪═════════╡
│   1    │     ppl      │     ppl     │  redshift   │    0    │
╘════════╧══════════════╧═════════════╧═════════════╧═════════╛
╒════════╤══════════════╤═════════════╤═════════════╤═════════╤═════════════╕
│  par#  │  Expression  │  Component  │  Parameter  │  Value  │    Prior    │
╞════════╪══════════════╪═════════════╪═════════════╪═════════╪═════════════╡
│   1*   │     ppl      │     ppl     │  $\alpha$   │   -1    │ unif(-2, 2) │
├────────┼──────────────┼─────────────┼─────────────┼─────────┼─────────────┤
│   2*   │     ppl      │     ppl     │ log$E_{p}$  │    2    │ unif(0, 4)  │
├────────┼──────────────┼─────────────┼─────────────┼─────────┼─────────────┤
│   3*   │     ppl      │     ppl     │   log$A$    │   -1    │ unif(-6, 5) │
╘════════╧══════════════╧═════════════╧═════════════╧═══

In [5]:
infer.par[1].frozen_at(-1.6)
infer.par[2].frozen_at(2.4)

print(infer.free_par_info)

╒════════╤══════════════╤═════════════╤═════════════╤═════════╤═════════════╕
│  par#  │  Expression  │  Component  │  Parameter  │  Value  │    Prior    │
╞════════╪══════════════╪═════════════╪═════════════╪═════════╪═════════════╡
│   3    │     ppl      │     ppl     │   log$A$    │   -1    │ unif(-6, 5) │
╘════════╧══════════════╧═════════════╧═════════════╧═════════╧═════════════╛



In [6]:
par_best = infer.minimize(method='Nelder-Mead')
infer.at_par(par_best)
infer.stat

4.352600856650524e-09

In [7]:
pair = infer.Pair[0]

bkg_cts = np.sum(pair.data.bkg_counts[0])
bkg_err = np.sqrt(np.sum(pair.data.bkg_errors[0] ** 2))
mo_cts = np.sum(pair.model.ctsrate[0] * pair.data.corr_src_exposure[0])

sigma = pgsig(mo_cts + bkg_cts, bkg_cts, bkg_err)
stat = (sigma - 3) ** 2 / 0.1

print('sigma:', sigma)
print('stat:', stat)

sigma: 2.9999340257561116
stat: 4.352600856650524e-08


In [8]:
print(model.par_info)

╒════════╤═════════════╤═════════════╤══════════╤═════════════╤══════════╕
│  par#  │  Component  │  Parameter  │  Value   │    Prior    │  Frozen  │
╞════════╪═════════════╪═════════════╪══════════╪═════════════╪══════════╡
│   1    │     ppl     │  $\alpha$   │   -1.6   │ unif(-2, 2) │   True   │
├────────┼─────────────┼─────────────┼──────────┼─────────────┼──────────┤
│   2    │     ppl     │ log$E_{p}$  │   2.4    │ unif(0, 4)  │   True   │
├────────┼─────────────┼─────────────┼──────────┼─────────────┼──────────┤
│   3    │     ppl     │   log$A$    │ -2.98188 │ unif(-6, 5) │  False   │
╘════════╧═════════════╧═════════════╧══════════╧═════════════╧══════════╛



In [9]:
model.ergflux(emin=10, emax=1000, ngrid=1000)

5.64988540706935e-08